# Extraction and classification of classical references from the ConDÉ corpus

Script written by Morgane Pica for a submission to the symposium ["Lire les classiques en Normandie"](https://rmblf.be/2022/02/04/appel-a-contribution-lire-les-classiques-en-normandie/) (oct 2022), to be written by herself and Mathieu Goux.

## Imports & declarations

In [13]:
from tqdm.notebook import tqdm #tqdm est bibliothèque qui permet d'avoir une barre de progression
import xml.etree.ElementTree as ET

ET.register_namespace("", "http://www.tei-c.org/ns/1.0")
ET.register_namespace('xml','http://www.w3.org/XML/1998/namespace')

# Not all witnesses were enriched with reference identification.
witnesses = ["basnage","berault","merville","pesnelle","terrien"]
binpath = "/home/mpica/Progs/perso/CONDE/editions/base-version/"
einpath = "_base.xml"

listfile = "authors.csv"
tablefile = "mentions.csv"

## Extract authors and store in a dictionary

In [35]:
authors = {}

def extract(witness, path):
    print(path)
    
    with open(path) as filein:
        tree = ET.parse(filein)
        root = tree.getroot()
        
        for author in tqdm(root.findall('.//{http://www.tei-c.org/ns/1.0}listPerson/{http://www.tei-c.org/ns/1.0}person')):
            fullbirth = author.find('.//{http://www.tei-c.org/ns/1.0}birth')
            
            try:
                ident = author.get("{http://www.w3.org/XML/1998/namespace}id")
                if ident not in authors.keys():
                    authors[ident] = {}
            
                    try:
                        if "when" in fullbirth.attrib.keys():
                            authors[ident]["birth"] = fullbirth.get("when")
                        else:
                            authors[ident]["birth"] = "?" + fullbirth.get("notBefore")
                            
                    except:
                        authors[ident]["birth"] = "none"
                        print(ident + " : no birth in " + witness)

                    try:
                        lg = {}
                        for name in author.findall('.//{http://www.tei-c.org/ns/1.0}persName'):
                            try:
                                lg[name.get("{http://www.w3.org/XML/1998/namespace}lang")] = name.find("./{http://www.tei-c.org/ns/1.0}surname").text + ", " + name.find("./{http://www.tei-c.org/ns/1.0}firstname").text
                            except:
                                lg[name.get("{http://www.w3.org/XML/1998/namespace}lang")] = name.text
                        if "fr" in lg.keys():
                            authors[ident]["name"] = lg["fr"]
                        elif "la" in lg.keys():
                            authors[ident]["name"] = lg["la"]
                        else:
                            authors[ident]["name"] = lg["en"]
                            
                    except:
                        authors[ident]["name"] = "none"
                        print(ident + " : no name in " + witness)
            except:
                continue
            

for witness in witnesses:
    fullpath = binpath + witness + einpath
    
    extract(witness, fullpath)

final = {}
for author in authors.keys():
    
    if "?" in authors[author]["birth"]:
        birth = authors[author]["birth"].replace("?","")
    try:
        if int(birth) < 550:
            final[author] = authors[author]
    except:
        print(authors[author])

print(final)

/home/mpica/Progs/perso/CONDE/editions/base-version/basnage_base.xml


  0%|          | 0/360 [00:00<?, ?it/s]

anian : no birth in basnage
herold : no birth in basnage
arq : no birth in basnage
goncanus : no birth in basnage
febur : no birth in basnage
s-michel : no birth in basnage
masurier : no birth in basnage
chartier : no birth in basnage
apulee : no birth in basnage
ald : no birth in basnage
frigentius : no birth in basnage
fillesac : no birth in basnage
isidore : no birth in basnage
adrien-pape : no birth in basnage
s-matthieu : no birth in basnage
fortin : no birth in basnage
adrien : no birth in basnage
antipater : no birth in basnage
moise : no birth in basnage
froben : no birth in basnage
s-thomas : no birth in basnage
josephe : no birth in basnage
balduin : no birth in basnage
s-jeanbap : no birth in basnage
josue : no birth in basnage
bannes : no birth in basnage
evariste : no birth in basnage
montholon : no birth in basnage
livius : no birth in basnage
bonneton : no birth in basnage
bam : no birth in basnage
colombel : no birth in basnage
pape-lucius : no birth in basnage
marcellu

  0%|          | 0/145 [00:00<?, ?it/s]

cassius : no birth in berault
angel : no birth in berault
nicephore : no birth in berault
muncer : no birth in berault
iudaus : no birth in berault
anchar : no birth in berault
galien : no birth in berault
timarchus : no birth in berault
/home/mpica/Progs/perso/CONDE/editions/base-version/merville_base.xml


...

/home/mpica/Progs/perso/CONDE/editions/base-version/pesnelle_base.xml


  0%|          | 0/207 [00:00<?, ?it/s]

skinner : no birth in pesnelle
lauriere : no birth in pesnelle
perard : no birth in pesnelle
robertson : no birth in pesnelle
gillet : no birth in pesnelle
camus : no birth in pesnelle
rigord : no birth in pesnelle
nicod : no birth in pesnelle
duplessis : no birth in pesnelle
chantreau : no birth in pesnelle
gousset : no birth in pesnelle
vedel : no birth in pesnelle
morgues : no birth in pesnelle
everard : no birth in pesnelle
renauldon : no birth in pesnelle
valentinien : no birth in pesnelle
bertheaume : no birth in pesnelle
maillard : no birth in pesnelle
gouget : no birth in pesnelle
avezan : no birth in pesnelle
/home/mpica/Progs/perso/CONDE/editions/base-version/terrien_base.xml


  0%|          | 0/92 [00:00<?, ?it/s]

{'pausanias': {'birth': '?0100', 'name': 'Pausanias'}, 'lampride': {'birth': '?0300', 'name': 'Lampride'}, 'bergier': {'birth': '1567', 'name': None}, 'andegav': {'birth': '1536', 'name': None}, 'connan': {'birth': '1508', 'name': None}, 'homere': {'birth': '?-0899', 'name': 'Homère'}, 'virgile': {'birth': '-0070', 'name': 'Virgile'}, 'goncanus': {'birth': 'none', 'name': 'Goncanus'}, 'bodin': {'birth': '1530', 'name': None}, 'martini': {'birth': '1614', 'name': None}, 'hotman': {'birth': '1524', 'name': None}, 'febur': {'birth': 'none', 'name': 'Febur.'}, 'bald': {'birth': '1327', 'name': 'Balde'}, 'herauld': {'birth': '1579', 'name': None}, 'pommeraye': {'birth': '1617', 'name': None}, 'caesar': {'birth': '-0100', 'name': None}, 's-germain': {'birth': '?0490', 'name': ' Germain (saint) '}, 'artemidore': {'birth': '?0100', 'name': "Artemidore d'Éphèse"}, 'turnebe': {'birth': '1512', 'name': None}, 'spartian': {'birth': '?0300', 'name': None}, 'const': {'birth': '0905', 'name': 'Consta